In [34]:
%matplotlib inline
import numpy as np
import os
import matplotlib.pyplot as plt
import tensorflow
import uuid
from azureml.core import Workspace
import azureml.core
import json
from azureml.core import Run, Experiment, Workspace, Model,Dataset
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.webservice import Webservice, AciWebservice,LocalWebservice
from azureml.core.authentication import AzureCliAuthentication


In [56]:

class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, (np.ndarray,np.number)):
            return obj.tolist()    
        print(obj)
        return json.JSONEncoder.default(self, obj)

In [57]:
cli_auth = AzureCliAuthentication()
ws = Workspace.from_config(auth=cli_auth)
with open('states/model_details.json', 'r') as f:
    model_details = json.load(f)
model = Model.deserialize(workspace=ws, model_payload=model_details['model'])


In [54]:
print(model)

Model(workspace=Workspace.create(name='imdbML-jazi-yacine', subscription_id='350ff211-0872-40d5-a629-9b672edbe452', resource_group='imdb-jazi-yacine'), name=imdb_model, id=imdb_model:1, version=1, tags={}, properties={})


In [58]:
env = Environment('imdb-deployment-env')
cd = CondaDependencies.create(
    pip_packages=['azureml-defaults','numpy', 'tensorflow']
)
env.python.conda_dependencies = cd
env.register(workspace = ws)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20211029.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "imdb-deployment-env",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-f

In [32]:

service = model.deploy(ws,
            name=service_name, 
            models=[model], 
            inference_config=inference_config, 
            deployment_config=aciconfig,
            overwrite=True)


In [60]:
env = Environment('imdb-training-env')
cd = CondaDependencies.create(
    pip_packages=['azureml-defaults','numpy', 'tensorflow']
)
env.python.conda_dependencies = cd
env.register(workspace = ws)
print()

In [46]:
print(ws)

Workspace.create(name='imdbML-jazi-yacine', subscription_id='350ff211-0872-40d5-a629-9b672edbe452', resource_group='imdb-jazi-yacine')


In [61]:
inference_config = InferenceConfig(entry_script='../steps/root/scripts/score.py', environment=env)

aciconfig = AciWebservice.deploy_configuration(
    cpu_cores=1, 
    memory_gb=1, 
)
service_name = 'imdb-recommendations-svc'# + str(uuid.uuid4())[:4]


#aciconfig = LocalWebservice.deploy_configuration(port=6778)
#service_name = 'imdb-recommendations-svc-' + str(uuid.uuid4())[:4]

service = Model.deploy(workspace=ws, 
                    name=service_name, 
                    models=[model], 
                    inference_config=inference_config, 
                    deployment_config=aciconfig,
                    overwrite=True)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-11-30 19:10:47+01:00 Creating Container Registry if not exists.
2021-11-30 19:10:47+01:00 Registering the environment.
2021-11-30 19:10:48+01:00 Use the existing image.
2021-11-30 19:10:48+01:00 Generating deployment configuration.
2021-11-30 19:10:49+01:00 Submitting deployment to compute.
2021-11-30 19:10:51+01:00 Checking the status of deployment imdb-recommendations-svc..
2021-11-30 19:16:46+01:00 Checking the status of inference endpoint imdb-recommendations-svc.
Failed


Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 532aec1f-cb47-4f8a-8407-7a3ae2b0029d
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Error in entry script, OSError: SavedModel file does not exist at: /var/azureml-app/azureml-models/imdb_model/{saved_model.pbtxt|saved_model.pb}, please run print(service.get_logs()) to get details.",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Error in entry script, OSError: SavedModel file does not exist at: /var/azureml-app/azureml-models/imdb_model/{saved_model.pbtxt|saved_model.pb}, please run print(service.get_logs()) to get details."
    }
  ]
}



WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 532aec1f-cb47-4f8a-8407-7a3ae2b0029d
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Error in entry script, OSError: SavedModel file does not exist at: /var/azureml-app/azureml-models/imdb_model/{saved_model.pbtxt|saved_model.pb}, please run print(service.get_logs()) to get details.",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Error in entry script, OSError: SavedModel file does not exist at: /var/azureml-app/azureml-models/imdb_model/{saved_model.pbtxt|saved_model.pb}, please run print(service.get_logs()) to get details."
    }
  ]
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Service deployment polling reached non-successful terminal state, current se

In [ ]:
with open("states/service_details.json", "w") as service_details:
    json.dump(service.serialize(), service_details)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Failed

Service deployment polling reached non-successful terminal state, current service state: Unhealthy
Operation ID: edd84005-8509-4a07-82df-ed39a10a05c2
Current sub-operation type not known, more logs unavailable.
Error:
{
  "code": "InaccessibleImage",
  "statusCode": 400,
  "message": "ACI Service request failed. Reason: The image 'imdbregistry.azurecr.io/azureml/azureml_70a72cb359999400f79db4858e3815d6' in container group 'imdb-recommendations-svc-57e6-lKxFSjjrKkS-ONjemfOGWg' is not accessible. Please check the image and registry credential.. Refer to https://docs.microsoft.com/azure/container-registry/container-registry-authentication#admin-account and make sure Admin user is enabled for your container registry."
}



WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Unhealthy
Operation ID: edd84005-8509-4a07-82df-ed39a10a05c2
Current sub-operation type not known, more logs unavailable.
Error:
{
  "code": "InaccessibleImage",
  "statusCode": 400,
  "message": "ACI Service request failed. Reason: The image 'imdbregistry.azurecr.io/azureml/azureml_70a72cb359999400f79db4858e3815d6' in container group 'imdb-recommendations-svc-57e6-lKxFSjjrKkS-ONjemfOGWg' is not accessible. Please check the image and registry credential.. Refer to https://docs.microsoft.com/azure/container-registry/container-registry-authentication#admin-account and make sure Admin user is enabled for your container registry."
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Service deployment polling reached non-successful terminal state, current service state: Unhealthy\nOperation ID: edd84005-8509-4a07-82df-ed39a10a05

In [68]:
print(ws)

Workspace.create(name='imdbML-jazi-yacine', subscription_id='350ff211-0872-40d5-a629-9b672edbe452', resource_group='imdb-jazi-yacine')


In [10]:
train_test_dataset = Dataset.get_by_name(ws, name='imdb_train_test')

test_data_path = os.path.join(os.getcwd(), 'data/imdb/test_data')
os.makedirs(test_data_path, exist_ok=True)

mnt_ctx_test_data = train_test_dataset.download(test_data_path,overwrite=True)
dataset_test = np.load(os.path.join(test_data_path, 'dataset_test.npy'))
dataset_train = np.load(os.path.join(test_data_path, 'dataset_train.npy'))



In [11]:
dataset_train[0]

array([1., 0., 0., ..., 0., 0., 0.])

In [12]:
print(dataset_test[0])

[1. 0. 0. ... 0. 0. 0.]


In [13]:
dumped = json.dumps({
    "data":np.expand_dims(dataset_test[0],0)
}, cls=NumpyEncoder)

response = service.run(dumped)
print(response['result'])

Checking container health...
Local webservice is running at http://localhost:6778
[[0.9128249883651733, 0.6529210805892944, 0.7423100471496582, 0.673879861831665, 0.7165325880050659, 0.7530237436294556, 0.8784433603286743, 0.6438800692558289, 0.3457186818122864, 0.7096614837646484, 0.8546161651611328, 0.48751482367515564, 0.7121099829673767, 0.7512127757072449, 0.4417855143547058, 0.4893118441104889, 0.868948221206665, 0.4042436182498932, 0.3567171096801758, 0.4625934958457947, 0.7408221364021301, 0.8158596754074097, 0.375171959400177, 0.7448878884315491, 0.5156201124191284, 0.7160546183586121, 0.8176919221878052, 0.8586812019348145, 0.6239693760871887, 0.5964089035987854, 0.8423506617546082, 0.5890745520591736, 0.6139043569564819, 0.834694504737854, 0.7047269940376282, 0.8409856557846069, 0.669814944267273, 0.3999684751033783, 0.8106489181518555, 0.8208602666854858, 0.716766357421875, 0.7088400721549988, 0.7744733095169067, 0.422006756067276, 0.44127875566482544, 0.7016855478286743, 0

{'result': [[0.8684630393981934, 0.6553806066513062, 0.7304263710975647, 0.6555782556533813, 0.7040827870368958, 0.7295724749565125, 0.845504879951477, 0.6511561274528503, 0.3477669954299927, 0.6667543053627014, 0.8040270209312439, 0.5266762971878052, 0.6893945932388306, 0.7104582190513611, 0.4239429831504822, 0.49340322613716125, 0.8326432704925537, 0.40317219495773315, 0.3641398847103119, 0.47343698143959045, 0.7245798110961914, 0.7746448516845703, 0.36793720722198486, 0.6983934640884399, 0.5249773859977722, 0.6857690811157227, 0.7702773809432983, 0.8409563302993774, 0.6284698843955994, 0.5701059103012085, 0.795769214630127, 0.5841270089149475, 0.5930420160293579, 0.8092092871665955, 0.6807385087013245, 0.8122087717056274, 0.6319349408149719, 0.4075700640678406, 0.7938229441642761, 0.7855349183082581, 0.7070623636245728, 0.6919110417366028, 0.7634838819503784, 0.42503800988197327, 0.4644638001918793, 0.669701874256134, 0.5425190925598145, 0.7131941914558411, 0.8011606931686401, 0.826

In [34]:
print(dataset_test[0])

[0. 0. 0. ... 0. 0. 0.]


<Response [500]>
